# Image Segmentation Predictor

In [ ]:
#Run this once per session
!pip install fastai -q --upgrade
!pip install git+https://github.com/WaterKnight1998/SemTorch # original install, pypi does not work!

  Cloning https://github.com/WaterKnight1998/SemTorch to /tmp/pip-req-build-esiwlfjv
  Running command git clone -q https://github.com/WaterKnight1998/SemTorch /tmp/pip-req-build-esiwlfjv


# Libraries

In [ ]:
from fastai.vision.all import *
import pandas as pd
import gc # garbage collector

In [ ]:
# everything happens in 'path' (except the model file)
path = Path("../Data/")

#### Choose class and other parameters:

In [ ]:
# should we use Nvidia CUDA?
use_CUDA = True

#Predict on testset or training?
Predict_TESTSET=True

# what class(es) are we predicting?
myclass = "aguada"
#myclass = "platform"
#myclass = "building"

# use TTA (test time augmentation)? It always never helped so far though...
use_TTA = False

############## TODO ### 
# where is my pre-trained model?
learner_path = "./models/"

# nnet model file name
nnet = "maya_{}_{}{}.pkl"

# path to images, i.e. which ones should be predicted?

if Predict_TESTSET:
    # FOR TESTSET:
    image_path = path/'lidar_test'
    #output_mask_path = path/'lidar_predicted_masks'
    # without threshold:
    output_mask_path = path/'lidar_predicted_masks_FULL'
else:
    # FOR TRAINING DATA:
    image_path = path/'lidar_train'
    output_mask_path = path/'lidar_predicted_masks_training_000'

myclass, learner_path, nnet, image_path, output_mask_path

('aguada',
 './models/',
 'maya_{}_{}{}.pkl',
 Path('../Data/lidar_test'),
 Path('../Data/lidar_predicted_masks_FULL'))

In [ ]:
# dummy function, just needs to exist, never invoked.
def segmentron_splitter(model):
    return [params(model.backbone), params(model.head)]

In [ ]:
### mask function dummy, just needs to exist, never invoked
path_lbl = path/'train_masks_normalized'
def get_msk(fn):
    return path_lbl/fn.name.replace("lidar", mymask)

In [ ]:
# here we collect all model *.pkl files of our class for ensembling their predictions
import glob
ensemble_list = glob.glob(learner_path+"*"+myclass+"*")
ensemble_list

['./models/maya_J040_aguada0.pkl',
 './models/maya_J040_aguada1.pkl',
 './models/maya_J040_aguada3.pkl',
 './models/stage-1 aguada1',
 './models/stage-1 aguada4',
 './models/maya_J040_aguada2.pkl',
 './models/stage-1 aguada0',
 './models/maya_J040_aguada4.pkl',
 './models/stage-1 aguada3',
 './models/stage-1 aguada2']

In [ ]:
# intermediate models are labelled "stage" and must be excluded:
ensemble_list = [m for m in ensemble_list if "stage" not in m ]

ensemble_list

['./models/maya_J040_aguada0.pkl',
 './models/maya_J040_aguada1.pkl',
 './models/maya_J040_aguada3.pkl',
 './models/maya_J040_aguada2.pkl',
 './models/maya_J040_aguada4.pkl']

In [ ]:
import gc

# create a testset and predict 
def predict(imagelist, myclass, ensemble_list, use_TTA=False):
    no_fold = len(ensemble_list) # number of available models to iterate over
    
    firstTime = True
    # we iterate over the ensembled models:
    for fold in ensemble_list:
        print ("---", fold)
        learn=load_learner(fold)
    
        if use_CUDA:# put it all to GPU
            dl_test = learn.dls.test_dl(imagelist).to('cuda')
            learn.model = learn.model.cuda()
            learn.dls.to('cuda')
        else:
            dl_test = learn.dls.test_dl(imagelist)

        if not use_TTA:
            predictions = learn.get_preds(dl=dl_test)  
        else:
            # with test time augmentation TTA. 
            # better results but takes much longer
            predictions = learn.TTA(dl=dl_test)
        
        # free some memory.
        del learn; gc.collect(); torch.cuda.empty_cache()
    
        # this is the ensembling, we compute the mean iteratively
        if firstTime:
            firstTime=False
            # initialization:
            preds = predictions[0]/no_fold
        else:
            # subsequently we add the other predictions and divide by the number of models
            # this results in computing the mean
            preds += predictions[0]/no_fold
       
        del predictions # free some memory.

    return preds

In [ ]:
# this stores the prediction as greyscale images. Predicted masks are scaled to [0..255] ubyte greyscale.
def save_predicted_masks_FULL(imagelist, preds, myclass):
    for i, pred in enumerate(preds):
        filename = imagelist[i].stem.replace('lidar', 'mask_') + myclass + '.tif'
        pred_scale = np.uint8((pred[0]*255).numpy())
        im = Image.fromarray(pred_scale)
        im.save(output_mask_path/filename)

In [ ]:
imagelist = get_image_files(image_path)
len(imagelist)

329

In [ ]:
# here we collect the predictions from the ensemmble
#%%time
preds = predict(imagelist, myclass, ensemble_list, use_TTA)

--- ./models/maya_J041_aguada0.pkl


/home/ju/miniconda3/envs/fastai/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


/home/ju/miniconda3/envs/fastai/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


--- ./models/maya_J040_aguada0.pkl


/home/ju/miniconda3/envs/fastai/lib/python3.7/site-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


...

--- ./models/maya_J041_aguada1.pkl


--- ./models/maya_J041_aguada2.pkl


--- ./models/maya_J041_aguada3.pkl


--- ./models/maya_J041_aguada4.pkl


CPU times: user 1min 29s, sys: 19.9 s, total: 1min 49s
Wall time: 1min 26s


In [ ]:
# ... and here we save it to disk as greyscale images
save_predicted_masks_FULL(imagelist, preds, myclass)

# END